In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 10 18:49:39 2018

@author: hugofayolle
"""

class Config:
    API_KEY = "WkHc4vJGHBT4Xtuma14T" # API key to access quandl
    MARKET = 'EURONEXT/'             # prefix of the code of a stock
    TAX = 0.0004                     # tax (percentage) applied when buying / selling stock
    FIGSIZE = (15,6)                 # dimensions of plots
    
class DevelopmentConfig(Config):
    SAVE_FOLDER = "/Users/hugofayolle/Desktop/Bourse/Bourse_test/Data/"
    TEST = True
    
class ProductionConfig(Config):
    SAVE_FOLDER = "/Users/hugofayolle/Desktop/Bourse/Bourse_test/Data/"
    TEST = False

config = DevelopmentConfig()


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 31 12:23:44 2018

@author: hugofayolle
"""

import os
import dill as pickle
import pandas as pd
import datetime


class File:
    def __init__(self, name, class_type = None, folder = config.SAVE_FOLDER, test = config.TEST):
        self.name = name
        if class_type is None:
            class_type = self.__class__.__name__
        self.type = class_type
        self.test = test
        if self.test:
            self.filename = "Test/" + self.type + "/" + self.name + ".p"
        else:
            self.filename = self.type + "/" + self.name + ".p"
        if folder[-1] != '/':
            folder = folder + '/'
        self.folder = folder
        self.path = self.folder + self.filename
        self.creation_date = pd.to_datetime(datetime.datetime.now().replace(microsecond = 0))
        self.last_update = self.creation_date
        
    def __repr__(self):
        if self.test:
            return self.type + " : " + self.name + " (test)"
        else:
            return self.type + " : " + self.name
        
    def drop(self):
        os.remove(self.path)

    def load(self):
        return pickle.load(open(self.path, "rb" ))

    def save(self, obj):
        pickle.dump(obj, open(self.path, "wb" ))
        self.last_update = pd.to_datetime(datetime.datetime.now().replace(microsecond = 0))
        
    def get_attributes(self):
        for (k,v) in self.__dict__.items():
            print(k + ' : ' + str(v))
    

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  7 12:02:02 2018

@author: hugofayolle
"""

import pandas as pd
import quandl

class Codes(File):
    def __init__(self):
        super().__init__('stock_infos')
        self.data = self.load()
        
    def add(self, code, full_name, description, first_date, include_in_analysis):
        if self.code_in_base(code):
            print("code " + code + " already in base, use self.modify if you want to change this value")
        else:
             # check if code is references in Quandl
            if not test_code(code):
                print('Code is not referenced in Quandl')
                print('Data not saved')
                return None
            # check date format
            first_date = check_date_format(first_date) 
            if first_date is None: return None
            # check if include_in_analysis is yes or no
            include_in_analysis = str(include_in_analysis).lower()
            if include_in_analysis != 'yes' and include_in_analysis != 'no':
                print("Please provide a 'yes' or 'no' value for parameter 'include_in_analysis'")
                print('Data not saved')
                return None
            self.data.loc[code]([full_name, description, first_date, include_in_analysis])
            self.data = self.data.sortindex()
            self.file.save(self.data)
            
    def change_first_date(self, code, new_first_date):
        new_first_date = check_date_format(new_first_date)
        if new_first_date is None:
            return None
        else:
            if not self.code_in_base(code):
                print('Code not in base')
                print('Data not saved')
            else:
                self.data.at[code, 'FIRST_DATE'] = new_first_date
                self.file.save(self.data)

    def change_analysis_status(self, code, new_status):
        if not code_in_base(self, code):
            print('Code not in base')
            print('Data not saved')
        else:
            self.data.at[code, 'INCLUDE_FOR_ANALYSIS'] = new_status
            self.file.save(self.data)
    
    def code_in_base(self, code):
        return code in list(self.data.index)
    
    def remove(self, code):
        if not self.code_in_base(code):
            print("code " + code + " not in base.")
        else:
            self.data = self.data.drop(code)
            self.file.save(self.data)

def check_date_format(date):
    try: 
        new_date = pd.to_datetime(date)
    except ValueError as v:
        print('Error occurred : ' + v.args[0])
        print('Data not saved')
        return None
    return new_date
     
def test_code(code):
    quandl.ApiConfig.api_key = config.API_KEY # mandatory to make api calls on quandl
    try:
        quandl.get(config.MARKET + code, start_date = config.START_DATE)
    except:
        return False
    return True

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 10 18:58:42 2018

@author: hugofayolle
"""

import datetime
import quandl                         # handles financial api
import matplotlib.pyplot as plt       # handles plots
import pandas as pd

class Stock(File):
    def __init__(self, code, test = config.TEST):
        codes = Codes()
        if not codes.code_in_base(code):
            print("Code " + code + " is not referenced in Quandl")
            print("Stock not created")
            return None
        super().__init__(name = code, test = test)
        self.data = pd.DataFrame()
        self.code = code
        self.full_code = config.MARKET + code
        self.full_name = codes.data.at[code, 'FULL_NAME']
        self.description = codes.data.at[code, 'DESCRIPTION']
        self.first_date = codes.data.at[code, 'FIRST_DATE']
        self.include_in_analysis = codes.data.at[code, 'INCLUDE_FOR_ANALYSIS']
        self.indicators = []
        self.strategies = []
        self.last_date = None
        self.save(self)
        
    def add_indicator(self, indicator):
        if indicator.name in self.indicators:
            print("Indicator " + indicator.name + " already applied on stock")
            print("Data not saved")
            return None
        indicator.apply_on_stock(self)
        self.indicators.append(indicator.name)
        self.save(self)
        
    def get_position(self, date):
        # returns the position of a given date within stock data
        return self.data.index.get_loc(date)
        
    def refresh(self, verbose = False, replace = False, to_date = None):
        # updates stock data with the latest data from quandl, computes indicators for the new values and computes orders for all strategies in Strategy
        if verbose: print("Updating stock : " + self.name + " values.")
        if replace: self.__init__(self.code, self.indicators, self.strategies)
        if not self.uptodate():
            df = self.get_data_from_api()[:to_date]
            self.data = pd.concat([self.data, df]).reset_index().drop_duplicates(subset='DATE', keep='first').set_index('DATE')
            self.last_date = self.data.iloc[-1].name
            
            for indicator_name in self.indicators:
                indicator = File(indicator_name, class_type = 'Indicator').load()
                indicator.apply_on_stock(self, verbose)
                
            """for strategy in self.strategies:
                strategy.apply_on_stock(self, verbose)
            self._set_last_refresh()"""
            
            self.save(self)
            print(self.code + " successfully updated!")
    
    def visualize(self, strategy = None, indicators = None, price = 'CLOSE'):
        if indicators is None:
            num_plots = 1
        else: # create as many plots as indicators
            m = max(indicators, key = indicators.get)
            num_plots = indicators[m]
            
        plot = Plot(num_plots)
        
        title = 'Stock : ' + self.name
        if strategy is not None:
            title = title + ', Strategy : ' + strategy.name
            
        plot.fill(self.data['DATE'], self.data[price], 1, title = title)
        
        if strategy is not None:
            df = strategy.trades(stock = self)
            for i in range(0, len(df)):
                opening_date = df.iat[i, 0] # index 0 for column is 'OPENING_DATE'
                plot.draw_vertical_line(opening_date)
                opening_index = list(self.data[self.data.DATE == opening_date].index)[0]
                closing_date = df.iat[i, 1]
                if str(closing_date) == 'NaT':
                    closing_index = self.last_row
                else:
                    closing_index = list(self.data[self.data.DATE == closing_date].index)[0]
                    plot.draw_vertical_line(closing_date)
                    
                sub_stock = self.data.iloc[opening_index:(closing_index+1), :]
                if df.iat[i, -1] >= 1:
                    color = 'green'
                else:
                    color = 'red'
                plot.fill(sub_stock['DATE'], sub_stock[price], 1, color)
                plot.set_xaxis(self.first_date, self.last_refresh)

        if indicators is not None:
            for (indicator, index) in indicators.items():
                plot.fill(self.data['DATE'], self.data[indicator.name], index)
                if strategy is not None:
                    for i in range(0, len(df)):
                        opening_date = df.iat[i, 0]
                        plot.draw_vertical_line(opening_date)
                        closing_date = df.iat[i, 1]
                        if str(closing_date) != 'NaT':
                            plot.draw_vertical_line(closing_date)
                plot.set_xaxis(self.first_date, self.last_refresh)
                
        plot.merge_xaxis()
        plt.show()
 
    def get_data_from_api(self):
        # returns dataframe from quandl api and formats it
        i = 0
        while i < 10:
            try:
                df = quandl.get(self.full_code, start_date=self.first_date)
            except Exception:
                i = i + 1
                if i == 10:
                    print('Max attempts reached')
                    print('Execution failed on stock : ' + self.code)
                    return None
                print('Error on stock : ' + self.code)
                print('Attempt ' + str(i))
                continue
            break
        df = df.reset_index()
        df = df.rename(index=str, columns={
            'Date':'DATE',
            'Open':'OPEN',
            'High':'HIGH',
            'Low':'LOW',
            'Last':'CLOSE',
            'Volume':'VOLUME',
            'Turnover':'TURNOVER'
        })
        df.DATE = pd.to_datetime(df.DATE)
        df = df.set_index('DATE')
        return df   
    
    def uptodate(self):
        if self.last_date is None: return False
        if self.test: return False
        last_update = pd.to_datetime(self.last_update.date())
        last_date = pd.to_datetime(self.last_date.date())
        today = pd.to_datetime(datetime.datetime.now().date())
        if last_update == today:
            return True
        elif today.weekday == 0:
            if (today-last_date).days == 3 and datetime.datetime.now().hour < 18:
                return True
        elif today.weekday == 5 and (today-last_date).days == 1:
            return True
        elif today.weekday == 6 and (today-last_date).days == 2:
            return True
        elif (today-last_date).days == 1 and datetime.datetime.now().hour < 18:
            return True
        else:
            return False 

class Indicator(File):
    """ This object contains all known technical indicators.
        It is defined by a function capable of computing the indicator on a given Series"""
    
    def __init__(self, name, compute, test = config.TEST):
        super().__init__(name = name, test = test)
        self.history = {}
        self.compute = compute
        self.save(self)
                
    def apply_on_stock(self, stock, verbose = False):
        # updates stock.data with computed value of indicator
        if verbose: print("Updating indicator " + self.name + " on stock " + stock.code + "...")
        if not self.name in stock.indicators: # if the indicator is not in stock data yet, creates it
            stock.data[self.name] = None
        stock.data[self.name] = self.compute(stock.data)
        self.history[stock.name] = stock.last_date
        self.save(self)
        if verbose: print("Indicator " + self.name + " successfully updated on stock " + stock.name + "!")
    
    def drop(self, stocks):
        for stock_name in list(self.history.keys()):
            stock = File(stock_name).load()
            stock.drop_column(self.name)
            stock.save(stock)
        super().drop(self)

import numpy as np

class Trade:
    def __init__(self, opening_date, closing_date, stock, status, bid_price, last_price):
        self.opening_date = opening_date
        self.closing_date = closing_date
        self.stock = stock.name
        self.status = status
        self.bid_price = bid_price
        self.last_price = last_price
        self.performance = self.get_performance()
        
    def __repr__(self):
        if self.performance is None:
            performance = " "
        else:
            performance = str(np.round((self.performance - 1)*100,3)) + "%"
        text = str(self.opening_date) + '\t'
        text = text + str(self.closing_date)
        if self.closing_date is None:
            text = text + "\t\t"
        text = text + "\t"
        text = text + str(self.stock) + '\t'
        text = text + self.status + '\t'
        if self.status != 'awaiting buy' and self.status != 'awaiting sell':
            text = text + '\t'
        text = text + str(self.bid_price) + '\t' \
                + str(self.last_price) + '\t' \
                + performance
        return text
            
    def get_performance(self):
        if self.bid_price is None:
            return None
        else:
            return self.last_price / self.bid_price * (1 - config.TAX) * (1 - config.TAX)
    
    def buy_awaiting(self):
        stock = File(self.stock, "Stock").load()
        position = stock.data.index.get_loc(self.opening_date)
        if position != len(stock.data):
            bid_price = stock.data.iloc[position + 1].OPEN
            self.set_bid_price(bid_price)
            self.set_opening_date(stock.data.iloc[position + 1].name)
            self.update_price(bid_price)
            self.change_status('pending')
    
    def sell_awaiting(self):
        stock = File(self.stock, "Stock").load()
        position = stock.data.index.get_loc(self.closing_date)
        if position != len(stock.data):
            self.close(stock.data.iloc[position + 1].name, stock.data.iloc[position + 1].OPEN)
            
    def awaiting_sell(self, last_price, closing_date):
        self.closing_date = closing_date
        self.update_price(last_price)
        self.change_status('awaiting sell')

    def close(self, closing_date, closing_price):
        self.closing_date = closing_date
        self.update_price(closing_price)
        self.status = 'closed'
        
    def set_bid_price(self, bid_price):
        self.bid_price = bid_price
    
    def set_opening_date(self, opening_date):
        self.opening_date = opening_date
        
    def change_status(self, status):
        self.status = status
    
    def get_key(self):
        return self.stock, self.status
    
    def update_price(self, last_price):
        self.last_price = last_price
        self.performance = self.get_performance()
                            
def open_trade(opening_date, stock, bid_price, status = 'pending'):
    return Trade(
            opening_date = opening_date,
            closing_date = None,
            stock = stock,
            status = status,
            bid_price = bid_price,
            last_price = bid_price)

class Function(File):
    def __init__(self, name, execute, test = config.TEST):
        super().__init__(name = name, test = test)
        self.name = name
        self.execute = execute
        self.save(self)

import pandas as pd

class Strategy(File):
    def __init__(self, name, buy_condition, sell_condition, description = '', test = config.TEST):
        super().__init__(name = name, test = test)
        self.description = description
        self.buy_condition = buy_condition
        self.sell_condition = sell_condition
        self.trades = []
        self.history = {}
        self.last_status = {}
        self.save(self)
    
    def print_trades(self):
        text = "Name : " + self.name + "\n"
        text = text + "Last refresh : " + str(self.last_update) + "\n"
        text = text + "OPENING_DATE\t\tCLOSING_DATE\t\tSTOCK\tSTATUS\t\tBID\tLAST\tPERF\n"
        for trade in self.trades:
            text = text + Trade.__repr__(trade) + "\n"
        print(text)
       
    def add_trade(self, trade):
        self.trades.append(trade)
        
    def drop_trade(self, trade):
        self.trades.remove(trade)
        self.save(self)
        
    def refresh(self, stocks, verbose = False):
        self.handle_awaiting_trades(stocks)
        for stock in stocks:
            if not self.name in stock.strategies:
                stock.strategies.append(self.name)
                stock.save(stock)
            if not self.contains(stock):
                self.initialize(stock)
            while self.history[stock.name] < stock.last_date:
                print(self.history[stock.name])
                data_to_update = stock.data[self.history[stock.name]:]
                iterator = data_to_update.itertuples()
                row = iterator.__next__()
                if self.last_status[stock.name] == 'awaiting buy':
                    self.order.sell(row, self.get_awaint)
                if self.last_status[stock.name] == 'pending':
                    trade = self.get_pending_trade(stock)
                    try:
                        sell_condition = self.sell_condition(stock, row, trade, iterator)
                        while not sell_condition['state']:
                            row = iterator.__next__()
                            sell_condition = self.sell_condition(stock, row, trade, iterator)
                        selling_date = sell_condition['date']
                        selling_price = sell_condition['price']
                        if selling_price is None:
                            selling_price = row.CLOSE
                            status = 'awaiting_sell'
                        else:
                            status = 'pending'
                        self.sell(trade = trade, date = selling_date, price = selling_price, status = status)
                    except StopIteration:
                        trade.update_price(row.CLOSE)
                elif self.last_status[stock.name] == 'closed':
                    try:
                        buy_condition = self.buy_condition(stock, row, iterator)
                        while not buy_condition['state']:
                            row = iterator.__next__()
                            buy_condition = self.buy_condition(stock, row, iterator)
                        buying_date = buy_condition['date']
                        buying_price = buy_condition['price']
                        if buying_price is None:
                            status = 'awaiting buy'
                        else:
                            status = 'pending'
                        self.buy(stock = stock, date = buying_date, price = buying_price, status = status)
                    except StopIteration:
                        pass
                elif self.last_status[stock.name] == 'pass':
                    try:
                        while self.buy_condition(stock, row, iterator)['state']:
                            row = iterator.__next__()
                        self.last_status[stock.name] = 'closed'
                    except StopIteration:
                        pass
                self.history[stock.name] = row.Index
        self.save(self)
        print('Trades for strategy ' + str(self.name) + ' successfully updated.')
    
    def buy(self, stock, date, price, status):
        trade = open_trade(opening_date = date, stock = stock, bid_price = price, status = status)
        self.last_status[stock.name] = status
        self.add_trade(trade)
    
    def sell(self, trade, date, price, status):
        if status == 'awaiting_sell':
            trade.awaiting_sell(last_price = price, closing_date = date)
            self.last_status[trade.stock]= 'awaiting_sell'
        else:
            trade.close(closing_date = date, closing_price = price)
            self.last_status[trade.stock] = 'pass'       
    
    def contains(self, stock):
        return stock.name in self.last_status.keys()

    def initialize(self, stock):
        self.history[stock.name] = stock.first_date
        self.last_status[stock.name] = 'pass'
    
    def handle_awaiting_trades(self, stocks):
        awaiting_buys, awaiting_sells = self.get_awaiting_trades(stocks)
        for trade in awaiting_buys:
            trade.buy_awaiting()
            self.last_status[trade.stock] = 'pending'
        for trade in awaiting_sells:
            trade.sell_awaiting()
            self.last_status[trade.stock] = 'pass'
        
    def get_awaiting_trades(self, stocks):
        awaiting_buys = [t for t in self.trades if t.status == 'awaiting buy' and t.stock in [s.name for s in stocks]]
        awaiting_sells = [t for t in self.trades if t.status == 'awaiting sell' and t.stock in [s.name for s in stocks]]
        return (awaiting_buys, awaiting_sells)
    
    def get_pending_trade(self, stock):
        return [t for t in self.trades if t.status == 'pending' and t.stock == stock.name][0]
    
    def get_closed_trades(self):
        return [t for t in self.trades if t.status == 'closed']
    
    def average_trade_duration(self):
        df = self.to_dataframe()
        df = df[df.STATUS == 'closed']
        return (df.CLOSING_DATE - df.OPENING_DATE).dt.days.sum() / len(df)
    
    def best(self, limit = 5):
        df = self.to_dataframe()
        return df.sort_values(by=["PERFORMANCE"], ascending=False)[:limit]
    
    def worst(self, limit = 5):
        df = self.to_dataframe()
        return df.sort_values(by=["PERFORMANCE"], ascending=True)[:limit]
    
    def won(self):
        return [t for t in self.trades if t.performance > 1]
    
    def lost(self):
        return [t for t in self.trades if t.performance < 1]
        
    def to_dataframe(self):
        df = pd.DataFrame(columns = ['OPENING_DATE', 'CLOSING_DATE', 'STOCK_NAME', 'STATUS', 'BID_PRICE', 'LAST_PRICE', 'PERFORMANCE'])
        i = 0
        for trade in self.trades:
            df.loc[i] = [trade.opening_date, trade.closing_date, trade.stock, trade.status, trade.bid_price, trade.last_price, trade.performance]
            i = i+1
        return df
    
    def summary(self):
        print(self.name + " : " + str(round(self.global_performance(), 2)) + "% in " + str(len(self.trades)) + " trades.")
        print("Average trade duration : " + str(self.average_trade_duration()) + " days.")
        print("Number of won trades : " + str(len(self.won())))
        print("Number of lost trades : " + str(len(self.lost())))
        print("Best trades")
        print(self.best().to_string())
        print("Worst trades")
        print(self.worst().to_string())
        print()
        
    def global_performance(self):
        df = self.to_dataframe()
        return df.PERFORMANCE.sum() - len(df)

import Model.exploration as e

In [80]:
orange = e.get_stock('ORA')

In [86]:
def buy(stock, row, iterator):
    date = None
    price = None
    status = None
    if np.isnan(row.SMA80) or np.isnan(row.SMA150):
        state = True
    else:
        state = row.SMA80 > row.SMA150
        if state == True:
            try:
                row = iterator.__next__()
                price = row.OPEN
                status = 'pending'
                date = row.Index
            except Exception:
                price = None
                status = 'awaiting buy'
                date = row.Index
    return {'state': state, 'date': date, 'price': price, 'status': status}

def sell(stock, row, trade, iterator):
    date = None
    price = None
    status = None
    state = row.SMA80 < row.SMA150
    if state == True:
        try:
            row = iterator.__next__()
            price = row.OPEN
            status = 'closed'
            date = row.Index
        except Exception:
            price = None
            status = 'awaiting sell'
            date = row.Index
    return {'state': state, 'date': date, 'price': price, 'status': status}

In [87]:
strategy = Strategy('SMA80_SMA150', buy, sell)

In [88]:
strategy.refresh([orange])

2014-02-14 00:00:00
2014-10-10 00:00:00
2014-12-04 00:00:00
2015-07-13 00:00:00
2015-07-13 00:00:00
2015-11-25 00:00:00
2016-05-04 00:00:00
2016-05-04 00:00:00
2017-01-09 00:00:00
2017-09-06 00:00:00
2017-09-06 00:00:00
2018-01-18 00:00:00
2018-04-24 00:00:00
2018-04-24 00:00:00
2018-06-12 00:00:00
2018-09-19 00:00:00
2018-09-19 00:00:00
Trades for strategy SMA80_SMA150 successfully updated.


In [89]:
strategy.trades

[2014-12-05 00:00:00	2015-07-14 00:00:00	ORA	closed		14.32	14.665	2.327%,
 2015-11-26 00:00:00	2016-05-05 00:00:00	ORA	closed		16.36	14.3	-12.662%,
 2017-01-10 00:00:00	2017-09-07 00:00:00	ORA	closed		14.8	13.91	-6.089%,
 2018-01-19 00:00:00	2018-04-25 00:00:00	ORA	closed		14.465	14.735	1.785%,
 2018-06-13 00:00:00	2018-09-20 00:00:00	ORA	closed		14.755	13.59	-7.969%]

In [91]:
e.plot_stock('ORA', strategy_name = 'SMA80_SMA150')

ModuleNotFoundError: No module named 'Model.ordertype'

In [41]:
while(a = 3):
    print(a)

SyntaxError: invalid syntax (<ipython-input-41-84866fc3412f>, line 1)